<a href="https://colab.research.google.com/github/zombienok/short_quest_generator/blob/main/quest_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/zombienok/short_quest_generator.git
!cd short_quest_generator
!pip install -r /content/short_quest_generator/requirements.txt
!curl -fsSL https://ollama.com/install.sh | sh

Cloning into 'short_quest_generator'...
remote: Enumerating objects: 136, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 136 (delta 55), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (136/136), 321.40 KiB | 3.38 MiB/s, done.
Resolving deltas: 100% (55/55), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 82.0 MB/s eta 0:00:00
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [2]:
import subprocess
import time
import ollama


#ИСПОЛЬЗУЕМАЯ МОДЕЛЬ
MODEL_TO_USE = "owl/t-lite:latest"

# Запускаем сервер Ollama в фоне (если ещё не запущен)
proc = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Ждём, пока сервер запустится (обычно 3-5 секунд)
time.sleep(1)

# Проверяем, работает ли сервер
try:
    ollama.pull(MODEL_TO_USE)
    print(ollama.list())  # Должен вывести список моделей (пустой, если ничего не загружено)
except Exception as e:
    print("Ошибка:", e)

models=[Model(model='owl/t-lite:latest', modified_at=datetime.datetime(2025, 8, 4, 20, 41, 27, 652556, tzinfo=TzInfo(UTC)), digest='52bcfdc27e1b605bb7905cfbfaca2cf5541e85f86c1e1138f0e00d11336242b1', size=4661229593, details=ModelDetails(parent_model='', format='gguf', family='llama', families=['llama'], parameter_size='8.0B', quantization_level='Q4_0'))]


**про константы для генерации структуры**

```COUNT_OF_FIRST_CHOICES = 2``` - количество выборов для стартовой сцены

```CHOICES_AVG = 2```  - максимальное количество выборов для всех остальных сцен

```DEPTH = 3``` - максимальная глубина

ориентировочное количество ```COUNT_OF_SCENES```сцен считается по  формуле:

```COUNT_OF_SCENES = COUNT_OF_FIRST_CHOICES *(CHOICES_AVG * DEPTH) +2```
, где +2 - стартовая сцена и финальная

Не рекомендуется менять значение констант (особенно увеличивать), т.к. даже незначительное увеличение приведёт к значительному увеличению количества сцен, что, соответственно, приведёт к переполнению контекста, галлюцинациям (не у Вас, к счастью) и неполной генерации.

In [3]:
# системный промпт для генерации структуры
messages_strict = [{
    "role": "system",
    "content": "Ты генератор структуры для квеста. Отвечаешь ТОЛЬКО в указанном формате. Никаких комментариев, пояснений, символов *, -, >."
}]

# КОНСТАНТЫ ДЛЯ ГЕНЕРАЦИИ СТРУКТУРЫ
COUNT_OF_FIRST_CHOICES = 2
CHOICES_AVG = 2
DEPTH = 3
COUNT_OF_SCENES = COUNT_OF_FIRST_CHOICES *(CHOICES_AVG * DEPTH) + 2

prompt_structure = f"""
Сгенерируй структуру квеста по следующим правилам:
1. Формат строго:
   <номер>. <scene_id>: <Выбор1> -> <target1>[, <Выбор2> -> <target2>]
2. Условия:
   - Минимум {COUNT_OF_SCENES} уникальных сцен
   - В стартовой сцене {COUNT_OF_FIRST_CHOICES} выбора
   - В нефинальных сценах может быть 1 или {CHOICES_AVG} выбора
   - Провалы (failX) могут появляться на любом этапе
   - Финали (finalX) только в конце веток
3. Пример:
   1. start_scene: 1 -> scene1, 2 -> scene2
   2. scene1: 1 -> scene3
   3. scene2: 1 -> fail1, 2 -> scene4
   4. scene3: 1 -> final1, 2 -> fail2
   5. scene4: 1 -> scene5
   6. scene5: 1 -> fail3, 2 -> final2
   7. scene6: 1 -> final3
   8. scene7: 1 -> fail4
"""
print(prompt_structure)
response_structure = ollama.chat(
    model=MODEL_TO_USE,
    messages=messages_strict + [{"role": "user", "content": prompt_structure}],
    options={"temperature":0.3} #ставим температуру пониже для предсказуемости результата
)
structure = response_structure["message"]["content"]
print(structure)


Сгенерируй структуру квеста по следующим правилам:
1. Формат строго:
   <номер>. <scene_id>: <Выбор1> -> <target1>[, <Выбор2> -> <target2>]
2. Условия:
   - Минимум 14 уникальных сцен
   - В стартовой сцене 2 выбора
   - В нефинальных сценах может быть 1 или 2 выбора
   - Провалы (failX) могут появляться на любом этапе
   - Финали (finalX) только в конце веток
3. Пример:
   1. start_scene: 1 -> scene1, 2 -> scene2
   2. scene1: 1 -> scene3
   3. scene2: 1 -> fail1, 2 -> scene4
   4. scene3: 1 -> final1, 2 -> fail2
   5. scene4: 1 -> scene5
   6. scene5: 1 -> fail3, 2 -> final2
   7. scene6: 1 -> final3
   8. scene7: 1 -> fail4

Конечно! Вот структура квеста по вашим правилам:

```
1. start_scene: 1 -> scene1, 2 -> scene2
2. scene1: 1 -> scene6, 2 -> scene3
3. scene2: 1 -> fail4, 2 -> final1
4. scene3: 1 -> fail5, 2 -> scene8
5. scene6: 1 -> final2, 2 -> fail7
6. scene8: 1 -> scene9, 2 -> fail6
7. scene9: 1 -> fail8, 2 -> final4
8. scene4: 1 -> scene10, 2 -> scene5
9. scene5: 1 -> fail

In [4]:
# извлекаем информацию из файла
# предполагается, что информация в файле заполнена по шаблону вида Cеттинг: ...\nГлавный герой: ...\nЦель: ...
def extract_info_from_txt(file_path):
    extracted_info = {
        "setting": None,
        "main_charachter": None,
        "goal": None
    }

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if line.startswith("Сеттинг:"):
                extracted_info["setting"] = line.split(":", 1)[1].strip()
            elif line.startswith("Главный герой:"):
                extracted_info["main_charachter"] = line.split(":", 1)[1].strip()
            elif line.startswith("Цель:"):
                extracted_info["goal"] = line.split(":", 1)[1].strip()

    return extracted_info


# импорируем данные из файла
FILE_PATH = r"/content/short_quest_generator/input/input_data3.txt"
extracted_info = extract_info_from_txt(FILE_PATH)
print(extracted_info)

json_strict = f"""
Преврати структуру квеста в JSON строго по этим правилам:
1. Сеттинг: {extracted_info["setting"]}
2. Главный герой: {extracted_info["main_charachter"]}
3. Финальная цель: {extracted_info["goal"]}
Правила генерации:
- Сохрани ВСЕ оригинальные scene_id из структуры и связи между сценами
- Для каждой сцены добавь:
  - "text": описание ситуации (2-3 предложения)
  - "choices": массив выборов (количество соответствует заданной ранее структуре)
- Требования к выборам:
  -выборы должны учитывать описание ГГ, окружающую обстановку и элемент случайности
  -Каждый выбор - конкретное действие (глагол + дополнение)
-"after": немедленное последствие выбора (1 предложение)
- Провалы (failX) должны быть в рамках сеттинга и ГГ
- Финалы (finalX) должны соотноситься с целью: """ + extracted_info["goal"] + """
- Провалы могут быть окончательными (полный провал квеста - "next_scene_id": "fail") или возвратными (возврат к какой-либо из предыдущих сцен - "next_scene_id": "scene1")
- значение next_scene_id для финальных сцен равно "end"
- значение next_scene_id для провальных сцен равно "GAME_OVER"
Формат строго JSON, где сцены выводятся через запятую, без комментариев. Пример формата одной сцены:
{
  "scene_id": "start_scene",
  "text": "...",
  "choices": [
      {
      "text": "...",
      "after": "...",
      "next_scene_id": "scene1"
      },
      {
      "text": "...",
      "after": "...",
      "next_scene_id": "scene2"
      }
    ]
}
"""
messages_strict_json = [{
        "role": "system",
        "content": json_strict}]
response_json = ollama.chat(
    model=MODEL_TO_USE,
    options = {"temperature": 0.7},
    messages=messages_strict_json + [{"role": "user", "content": structure}],
    format = "json"
)

print(response_json["message"]["content"])

{'setting': 'Постапокалиптическая Сибирь. Метели не стихают уже 12 лет. Вода дороже нефти, по земле бродят разумные мутанты, торгующие артефактами прошлого.', 'main_charachter': 'Бывший учёный-биолог, ныне — охотник за реликтами. Потерял руку в стычке с ледяным червём, теперь носит протез, работающий на радиации. Обожает слушать аудиозаписи дореволюционного шансона.', 'goal': 'Проникнуть в заброшенный техно-бункер, где спрятан образец "Зелёного кода" — вируса, способного вернуть экосистеме способность к регенерации. Но сначала надо договориться с мутантами-хранителями, пройти через поле аномалий и не сойти с ума от шёпота стен.'}
{
    "start_scene": {
        "scene_id": "start_scene",
        "text": "Сибирская метель стихнет только через день, но вам повезло: в заброшенном бункере есть теплый уголок и кое-какие припасы. Вы замечаете артефакт на столе – старая карта с непонятными отметками.",
        "choices": [
            {
                "text": "Внимательно изучить карту (1).",

In [5]:
import json
import re

str_json = response_json["message"]["content"]
# Извлечение JSON (на случай, если модель "добавила мусор")
quest_json = json.loads(
    re.search(r'\{.*\}', str_json, flags=re.DOTALL).group(0)
)

print(quest_json)  # Готовый JSON для работы

{'start_scene': {'scene_id': 'start_scene', 'text': 'Сибирская метель стихнет только через день, но вам повезло: в заброшенном бункере есть теплый уголок и кое-какие припасы. Вы замечаете артефакт на столе – старая карта с непонятными отметками.', 'choices': [{'text': 'Внимательно изучить карту (1).', 'after': 'Открываешь старые надписи и находишь следы, ведущие к местам активности мутантов. Ты знаешь, что это может быть опасно.', 'next_scene_id': 'scene1'}, {'text': 'Проверить припасы (2).', 'after': 'Время на разборку снаряжения и оценке рациона уходит, но теперь ты готов к любой непредвиденной ситуации.', 'next_scene_id': 'scene6'}]}, 'scene1': {'scene_id': 'scene1', 'text': 'Следы ведут в пустошь. Вы решаете пойти к ближайшему мутанту и попросить помощи.', 'choices': [{'text': 'Постучать по входному люку (1).', 'after': 'Люки оказываются запертыми, но через минуту открывается окошко. Огромный мутант смотрит на тебя и требует опознаться.', 'next_scene_id': 'scene2'}, {'text': 'Попро

In [6]:
# Запись строки в файл
with open("quest.json", "w", encoding="utf-8") as file:
    json.dump(quest_json, file, ensure_ascii=False, indent=4)

# Визуализация (опционально, нестабильно)

In [23]:
from graphviz import Digraph
import json
import re
from google.colab import files
from IPython.display import display, HTML

def safe_get(d, keys, default=None):
    for key in keys:
        if isinstance(d, dict) and key in d:
            d = d[key]
        else:
            return default
    return d if d is not None else default

def extract_scenes(data):
    # Если start_scene — это объект, добавим его явно
    scenes = {}
    for k, v in data.items():
        if isinstance(v, dict) and 'scene_id' in v:
            scenes[v['scene_id']] = v
    return scenes

def build_scene_graph(scenes, start_id):
    dot = Digraph(format='png')
    visited = set()

    def walk(scene_id):
        if scene_id in visited or scene_id not in scenes:
            return
        visited.add(scene_id)

        scene = scenes[scene_id]
        text = safe_get(scene, ['text'], '')[:60].replace('\n', ' ')
        dot.node(scene_id, f"{scene_id}\n{text}", shape='box', style='filled',
                 fillcolor='lightblue' if scene_id == start_id else 'white')

        for choice in scene.get('choices', []):
            target = choice.get('next_scene_id')
            if target:
                label = choice.get('text', '...')[:40]
                dot.edge(scene_id, target, label=label)
                walk(target)

        # Поддержка одиночных переходов вне choices
        next_scene_id = scene.get('next_scene_id')
        if next_scene_id:
            dot.edge(scene_id, next_scene_id)
            walk(next_scene_id)

    walk(start_id)
    return dot

def load_json_content():
    print("1: Загрузить файл\n2: Указать путь\n3: Использовать '/content/quest.json'")
    option = input("Выберите способ (1/2/3): ").strip()
    try:
        if option == '1':
            uploaded = files.upload()
            if not uploaded:
                raise ValueError("Файл не загружен.")
            return list(uploaded.values())[0].decode('utf-8')
        elif option == '2':
            path = input("Введите путь: ").strip()
            with open(path, 'r', encoding='utf-8') as f:
                return f.read()
        elif option == '3':
            with open('/content/quest.json', 'r', encoding='utf-8') as f:
                return f.read()
        else:
            raise ValueError("Некорректный выбор.")
    except Exception as e:
        print(f"Ошибка загрузки: {e}")
        return None

def visualize_tree():
    content = load_json_content()
    if not content:
        return

    try:
        data = json.loads(content)
    except json.JSONDecodeError:
        print("Попытка очистить JSON от лишних запятых...")
        content = re.sub(r",\s*([\]}])", r"\1", content)
        data = json.loads(content)

    scenes = extract_scenes(data)
    start = data.get('start_scene')
    start_id = start.get('scene_id') if isinstance(start, dict) else start
    if start_id not in scenes:
        print(f"Стартовая сцена '{start_id}' не найдена.")
        return

    dot = build_scene_graph(scenes, start_id)
    display(HTML(dot.pipe(format='svg').decode('utf-8')))

visualize_tree()


1: Загрузить файл
2: Указать путь
3: Использовать '/content/quest.json'
Выберите способ (1/2/3): 3
